In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 10

gpu_id='gpu0'
import os
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"

import numpy as np
from keras.layers.advanced_activations import SReLU
from keras import callbacks as ckbs
import random
import time
import gzip
from keras.models import model_from_json
import pickle as pkl
import keras
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.wrappers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from docutils.languages.af import labels
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder



Autosaving every 10 seconds


Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


In [2]:
def load_neural_network(file_from):
    (nn_arch, nn_weights_path) = pkl.load(open(file_from, 'rb'))
    nn = model_from_json(nn_arch)
    nn.set_weights(nn_weights_path)
    return nn

In [3]:
f = gzip.open('../data/processed/pkl_short/embeddings.pkl.gz', 'rb')
embeddings = pkl.load(f)
f.close()

label2Idx = embeddings['label2Idx']

#Inverse label mapping
idx2Label = {v: k for k, v in label2Idx.items()}

f = gzip.open('../data/processed/pkl/data.pkl.gz', 'rb')
test_data = pkl.load(f)
dev_data = pkl.load(f)
wiki_data = pkl.load(f)
f.close()

n_out = len(label2Idx)

In [4]:
class FullModelCheckpoint(ckbs.ModelCheckpoint):
    def on_epoch_end(self, epoch, logs={}):
        filepath = self.filepath.format(epoch=epoch, **logs)
        if self.save_best_only:
            current = logs.get(self.monitor)
            if current is None:
                warnings.warn('Can save best model only with %s available, '
                              'skipping.' % (self.monitor), RuntimeWarning)
            else:
                if self.monitor_op(current, self.best):
                    if self.verbose > 0:
                        print('Epoch %05d: %s improved from %0.5f to %0.5f,'
                              ' saving model to %s'
                              % (epoch, self.monitor, self.best,
                                 current, filepath))
                    self.best = current
                    pkl.dump([self.model.to_json(), self.model.get_weights()], open(filepath, 'wb'))
                else:
                    if self.verbose > 0:
                        print('Epoch %05d: %s did not improve' %
                              (epoch, self.monitor))
        else:
            if self.verbose > 0:
                print('Epoch %05d: saving model to %s' % (epoch, filepath))
            pkl.dump([self.model.to_json(), self.model.get_weights()], open(filepath, 'wb'))


In [5]:
def get_model(matrix, matrix_test):
    emb_layers = []
    for param_num in range(matrix.shape[2]):
        model = Sequential()
        emd_max1 = np.max(matrix[:, :, param_num]) + 2
        emd_max = np.max(matrix_test[:, :, param_num]) + 2
        if emd_max > emd_max1:
            embedding_max_index = emd_max
        else:
            embedding_max_index = emd_max1
        out_size = 3
        if embedding_max_index > 300:
            out_size = 300
        elif embedding_max_index > 100:
            out_size = 10
        model.add(Embedding(embedding_max_index, out_size, input_length=matrix.shape[1]))
        emb_layers.append(model)
    
    merged = Merge(emb_layers, mode = 'concat')
    return merged
    

In [6]:
def get_data(data):
    print( "%d test size" % len(data))
    data = np.array(data)
    print(data.shape)
    X = [x[0] for x in data]
    y = [x[1][0] for x in data]
    return np.array(X), np.array(y)
    

In [7]:
def matrix_train_val_test_split(matrix, target, number_of_samples_to_use=None, test_size=0.2, random_state=23):
   
    train_samples = int(matrix.shape[0]*1)
    X_train = matrix[:train_samples]
    X_test = matrix[train_samples:]
        
    y_train = target[:train_samples]
    y_test = target[train_samples:]
        
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size, random_state=random_state)
    print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)
    
    
    return  [X_train[:,:,i] for i in range(X_train.shape[2])], \
            [X_val[:,:,i] for i in range(X_val.shape[2])], \
            y_train, y_val

In [22]:
def get_full_model(X, X_test, name):
    merged = get_model(X, X_test)
    model = Sequential()
    model.add(merged)
    #model.add(LSTM(100, return_sequences=True, dropout_W=0.2))
    #model.add(LSTM(100, return_sequences=True, dropout_W=0.2))
    model.add(LSTM(100, dropout_W=0.25))
    #model.add(Dropout(0.2))
    #model.add(Dense(100))
    #model.add(SReLU())
    model.add(Dense(n_out, activation='softmax'))

    sgd = SGD(lr=0.1, decay=1e-7, momentum=0.0, nesterov=False, clipvalue=3) 
    rmsprop = RMSprop(clipvalue=3) 
    nadam = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    model.compile(loss='categorical_crossentropy', optimizer=nadam,metrics=['accuracy', 'precision', 'recall', 'categorical_accuracy', 'fmeasure'])

    #model.summary()
    checkpointer = FullModelCheckpoint(filepath= "../models/" + name, verbose=1, save_best_only=True)
    return model, checkpointer


In [9]:
X,y = get_data(dev_data)
X_test, y_test = get_data(test_data)

31017 test size
(31017, 2)
59381 test size
(59381, 2)


In [10]:
X_train, X_val, y_train, y_val = matrix_train_val_test_split(X, y)
y_train_dum = OneHotEncoder(sparse=False).fit_transform(y_train.reshape(-1, 1))
y_val_dum = OneHotEncoder(sparse=False).fit_transform(y_val.reshape(-1, 1))

(24813, 33, 25) (6204, 33, 25) (0, 33, 25) (24813,) (6204,) (0,)


In [23]:
model, checkpointer = get_full_model(X, X_test, "new5")

In [24]:

model.fit(X_train, y_train_dum, batch_size=32, nb_epoch=11, validation_data=(X_val, y_val_dum), callbacks=[checkpointer])

Train on 24813 samples, validate on 6204 samples
Epoch 1/11
24813/24813 [==============================] - 56s - loss: 0.4333 - acc: 0.8721 - precision: 0.8813 - recall: 0.8586 - categorical_accuracy: 0.8721 - fmeasure: 0.8692 - val_loss: 0.3499 - val_acc: 0.8851 - val_precision: 0.8931 - val_recall: 0.8749 - val_categorical_accuracy: 0.8851 - val_fmeasure: 0.8838
Epoch 2/11
24813/24813 [==============================] - 56s - loss: 0.1677 - acc: 0.9402 - precision: 0.9482 - recall: 0.9325 - categorical_accuracy: 0.9402 - fmeasure: 0.9401 - val_loss: 0.1544 - val_acc: 0.9513 - val_precision: 0.9564 - val_recall: 0.9473 - val_categorical_accuracy: 0.9513 - val_fmeasure: 0.9517
Epoch 3/11
24813/24813 [==============================] - 56s - loss: 0.0314 - acc: 0.9899 - precision: 0.9903 - recall: 0.9891 - categorical_accuracy: 0.9899 - fmeasure: 0.9897 - val_loss: 0.1621 - val_acc: 0.9478 - val_precision: 0.9488 - val_recall: 0.9462 - val_categorical_accuracy: 0.9478 - val_fmeasure: 0.94